# Comparação de Modelos: CNN vs. LSTM 🥊

Neste notebook, treinamos as duas arquiteturas com os mesmos dados e hiperparâmetros para comparar o desempenho.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import sys
import os

# Adicionar diretório raiz ao path para importar módulos
sys.path.append(os.path.abspath(".."))

from utils.utils import url_to_tensor, VOCAB_SIZE
from models.models import CharCNN, CharLSTM
from train import URLDataset, train_model, evaluate_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo: {device}")

In [ ]:
# Configurações
BATCH_SIZE = 64
EPOCHS = 5
LR = 0.001
MAX_LEN = 150

# Carregar Dados (Amostra para teste rápido, remova .sample() para treino completo)
df = pd.read_csv("../data/malicious_phish.csv") #.sample(50000, random_state=42)

# Mapeamento binário simples
df['label'] = df['type'].apply(lambda x: 0 if x == 'benign' else 1)

X_train, X_test, y_train, y_test = train_test_split(df['url'].values, df['label'].values, test_size=0.2, random_state=42)

train_loader = DataLoader(URLDataset(X_train, y_train, MAX_LEN), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(URLDataset(X_test, y_test, MAX_LEN), batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
def run_experiment(model_class, name):
    print(f"\n--- Treinando {name} ---")
    model = model_class(vocab_size=VOCAB_SIZE, num_classes=2).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LR)
    
    history = {'train_acc': [], 'test_acc': [], 'train_loss': []}
    
    for epoch in range(EPOCHS):
        t_loss, t_acc = train_model(model, train_loader, criterion, optimizer)
        _, v_acc = evaluate_model(model, test_loader, criterion)
        
        history['train_acc'].append(t_acc)
        history['test_acc'].append(v_acc)
        history['train_loss'].append(t_loss)
        print(f"Ep {epoch+1}: Train Acc {t_acc:.2f}% | Test Acc {v_acc:.2f}%")
        
    return history

# Executar Experimentos
cnn_history = run_experiment(CharCNN, "CharCNN")
lstm_history = run_experiment(CharLSTM, "CharLSTM")

In [ ]:
# Plotar Resultados Comparativos
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(cnn_history['test_acc'], label='CNN', marker='o')
plt.plot(lstm_history['test_acc'], label='LSTM', marker='s')
plt.title('Acurácia de Teste por Época')
plt.xlabel('Época')
plt.ylabel('Acurácia (%)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(cnn_history['train_loss'], label='CNN', linestyle='--')
plt.plot(lstm_history['train_loss'], label='LSTM', linestyle='--')
plt.title('Perda de Treinamento (Loss)')
plt.xlabel('Época')
plt.legend()

plt.tight_layout()
plt.show()